240.661

vns
 Athletes: ['lukas dauser', 'nils dunkel', 'pascal brendel', 'andreas toba']
 Score: 248.19400000000002

hh
Athletes: ['glenn trebing' 'dario sissakis' 'lukas dauser' 'pascal brendel']
 Score: 244.027
 
ts
Athletes['pascal brendel' 'lukas dauser' 'nils dunkel' 'andreas toba']
 Score:
248.19400000000002

In [22]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')


In [3]:
df=df[df['Nation']=='Germany']

In [5]:
df=df.drop(columns=['Rank', 'Nation', 'round', 'year', 'AA'])

In [6]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values
})


vns
Selected Athletes: ['lukas dauser', 'nils dunkel', 'pascal brendel', 'andreas toba']
Total Score: 248.19400000000002

In [7]:
df = df_max_scores.copy()

In [8]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']  

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(6000): 
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['Athlete'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


Selected Athletes: ['lukas dauser', 'nils dunkel', 'pascal brendel', 'andreas toba']
Total Score: 248.19400000000002
Best Seed: 411


hh
Best Selection: ['glenn trebing' 'dario sissakis' 'lukas dauser' 'pascal brendel']
Best Score: 244.027


In [9]:
df = df_max_scores.copy()

In [10]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['glenn trebing' 'dario sissakis' 'lukas dauser' 'pascal brendel']
Best Score: 244.027


ts

In [11]:
df = df_max_scores.copy()

In [12]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 248.19400000000002
Iteration 2, Best Score: 248.19400000000002
Iteration 3, Best Score: 248.19400000000002
Iteration 4, Best Score: 248.19400000000002
Iteration 5, Best Score: 248.19400000000002
Iteration 6, Best Score: 248.19400000000002
Iteration 7, Best Score: 248.19400000000002
Iteration 8, Best Score: 248.19400000000002
Iteration 9, Best Score: 248.19400000000002
Iteration 10, Best Score: 248.19400000000002
Iteration 11, Best Score: 248.19400000000002
Iteration 12, Best Score: 248.19400000000002
Iteration 13, Best Score: 248.19400000000002
Iteration 14, Best Score: 248.19400000000002
Iteration 15, Best Score: 248.19400000000002
Iteration 16, Best Score: 248.19400000000002
Iteration 17, Best Score: 248.19400000000002
Iteration 18, Best Score: 248.19400000000002
Iteration 19, Best Score: 248.19400000000002
Iteration 20, Best Score: 248.19400000000002
Iteration 21, Best Score: 248.19400000000002
Iteration 22, Best Score: 248.19400000000002
Iteration 23, Best 

In [13]:
import pandas as pd

data = {
    'Method': ['vns', 'hh', 'ts'],
    'Athletes': [
        ['lukas dauser', 'nils dunkel', 'pascal brendel', 'andreas toba'],
        ['glenn trebing', 'dario sissakis', 'lukas dauser', 'pascal brendel'],
        ['pascal brendel', 'lukas dauser', 'nils dunkel', 'andreas toba']
    ],
    'Score': [248.194, 244.027, 248.194]
}

df = pd.DataFrame(data)



In [14]:
df

,Method,Athletes,Score
0,vns,"[lukas dauser, nils dunkel, pascal brendel, an...",248.194
1,hh,"[glenn trebing, dario sissakis, lukas dauser, ...",244.027
2,ts,"[pascal brendel, lukas dauser, nils dunkel, an...",248.194


In [15]:
# df.to_csv('de.csv', index=False)

In [28]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')
df=df[df['Nation']=='Germany']
df=df.drop(columns=['Rank', 'Nation', 'round', 'year'])
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})


In [29]:
df=df_max_scores.copy()

In [30]:
df=df.sort_values(by='AA', ascending=False)

In [31]:
df

,Athlete,VT,PH,SR,FX,PB,HB,AA
7,lukas dauser,13.866,13.233,13.133,13.666,15.400,13.566,82.364
10,pascal brendel,13.633,13.300,13.200,14.000,13.666,13.933,80.066
5,glenn trebing,13.766,12.166,13.166,12.833,14.400,13.100,78.231
2,carlo hörr,14.000,11.300,12.266,13.566,13.366,12.000,76.498
6,lucas kochan,14.066,13.100,0.000,12.900,14.233,13.400,67.699
9,nils dunkel,0.000,14.600,13.533,13.166,14.133,12.500,54.931
1,andreas toba,14.300,13.433,13.666,0.000,0.000,13.133,54.532
8,nick klessing,14.433,0.000,13.733,12.233,0.000,0.000,40.399
3,dario sissakis,14.300,12.266,0.000,13.633,0.000,0.000,40.199
4,felix remuta,12.900,0.000,0.000,13.700,13.333,0.000,39.933


In [32]:
82.364+80.066+ 	78.231

240.661